In [1]:
from harmony_tokenizers_m21 import ChordSymbolTokenizer, MelodyPitchTokenizer, MergedMelHarmTokenizer

chordSymbolTokenizer = ChordSymbolTokenizer.from_pretrained('saved_tokenizers/ChordSymbolTokenizer')
#melodyPitchTokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')

#m_chordSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, chordSymbolTokenizer, verbose=1)

xml_file_path = './data/results/xmls/bart_0.8/ChordSymbolTokenizer/mxl_0000_gen.mxl'
#toks_symb_m = m_chordSymbolTokenizer.encode(xml_file_path)

midi_file_path = './data/results/midis/bart_0.8/ChordSymbolTokenizer/mid_0002_real.mid'

harm_tokens = chordSymbolTokenizer.encode(xml_file_path,add_start_harmony_token=False)['input_tokens'][1:]
# Filter out tokens that start with '<' or 'position'
chord_tokens = [token for token in harm_tokens if not (token.startswith('<') or token.startswith('position'))]
print(chord_tokens)

#create all chord vocab
all_chord_vocab =  [token for token, idx in chordSymbolTokenizer.vocab.items() if idx >= 88]


['C:maj', 'C:maj', 'F:maj', 'C:maj', 'C:maj', 'C:maj', 'F:maj', 'C:maj']


C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\music21\stream\base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [2]:
import os
import glob
from collections import Counter
from music21 import converter, key

def detect_midi_key(midi_file_path):
    """
    Detect the key (major or minor) of a MIDI file using music21.
    
    Parameters:
        midi_file_path (str): The path to the MIDI file.
        
    Returns:
        str: A string representing the key, e.g., "C major" or "A minor".
             Returns None if detection fails.
    """
    try:
        # Parse the MIDI file into a music21 stream.
        score = converter.parse(midi_file_path)
        # Use music21's built-in key analysis.
        key_obj = score.analyze('key')
        detected_key = f"{key_obj.tonic.name} {key_obj.mode}"
        return detected_key
    except Exception as e:
        print(f"Error detecting key from {midi_file_path}: {e}")
        return None

# Replace with the path to your folder containing the MIDI files.
folder_path = "./data/results/midis/gpt_1.0/ChordSymbolTokenizer"
pattern = os.path.join(folder_path, "*_real.mid")
midi_files = glob.glob(pattern)

# Use a Counter to store counts for each detected key.
key_counts = Counter()

for midi_file_path in midi_files:
    key_signature = detect_midi_key(midi_file_path)
    if key_signature:
        key_counts[key_signature] += 1

# Print the counts for each key.
print("Detected key counts:")
for key_sig, count in key_counts.items():
    print(f"{key_sig}: {count}")


KeyboardInterrupt: 

In [13]:
import os
import glob
import numpy as np
import pretty_midi

#extract chords from midi
midi_file_path = './data/results/midis/bart_0.8/ChordSymbolTokenizer/mid_0000_real.mid'

def extract_chords_with_times_from_midi(midi_file_path, track_index=1, grouping_threshold=0.05):
    """
    Extract chords from the specified track of a MIDI file along with their onset times.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        track_index (int): Track index for chords.
        grouping_threshold (float): Time threshold to group notes.
    
    Returns:
        List[Tuple[float, Tuple[int]]]: A list of (onset_time, chord) pairs.
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    if track_index >= len(pm.instruments):
        raise ValueError(f"Track index {track_index} is out of range for file {midi_file_path}.")
    
    chord_instrument = pm.instruments[track_index]
    sorted_notes = sorted(chord_instrument.notes, key=lambda note: note.start)
    
    chords_with_times = []
    current_chord_notes = []
    current_time = None

    for note in sorted_notes:
        if current_time is None:
            current_time = note.start
            current_chord_notes.append(note)
        else:
            if note.start - current_time <= grouping_threshold:
                current_chord_notes.append(note)
            else:
                chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
                chords_with_times.append((current_time, chord_pitch_classes))
                current_chord_notes = [note]
                current_time = note.start

    if current_chord_notes:
        chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
        chords_with_times.append((current_time, chord_pitch_classes))
    
    return chords_with_times


def compute_HRHE_and_HRC(chords_with_times, quantization=0.05):
    """
    Compute the Harmonic Rhythm Histogram Entropy (HRHE) and Harmonic Rhythm Coverage (HRC)
    from a list of chord events with onset times.
    
    Parameters:
        chords_with_times (List[Tuple[float, chord]]): List of tuples where each tuple contains
            the onset time (in seconds) and a chord (represented as pitch classes).
        quantization (float): Quantization step (in seconds) used to discretize the inter-chord intervals.
    
    Returns:
        tuple: (HRHE, HRC)
            HRHE (float): The entropy (in nats) of the histogram of quantized inter-chord intervals.
            HRC (int): The number of unique quantized interval types.
    """
    if len(chords_with_times) < 2:
        return 0.0, 0

    # Compute inter-onset intervals between successive chord events.
    intervals = []
    for i in range(1, len(chords_with_times)):
        diff = chords_with_times[i][0] - chords_with_times[i-1][0]
        # Quantize the interval (e.g., rounding to the nearest multiple of 'quantization')
        quantized_diff = round(diff / quantization) * quantization
        intervals.append(quantized_diff)
    print(intervals)
    
    # Build a histogram (frequency count) of the quantized intervals.
    hist = {}
    for diff in intervals:
        hist[diff] = hist.get(diff, 0) + 1

    total = sum(hist.values())
    HRHE = 0.0
    for count in hist.values():
        p = count / total
        HRHE -= p * np.log(p + 1e-6)  # using a small epsilon to avoid log(0)
    
    # HRC: number of unique quantized interval types.
    HRC = len(hist)
    return HRHE, HRC


chords_with_times = extract_chords_with_times_from_midi(midi_file_path)
HRHE, HRC = compute_HRHE_and_HRC(chords_with_times)
print('HRHE:', HRHE)
print('HRC:', HRC)


from music21 import converter, meter
import numpy as np

def compute_CBS(midi_file_path, chord_part_index=1, tolerance=0.05):
    """
    Compute the Chord Beat Strength (CBS) for a MIDI file.
    
    This implementation uses music21 to extract measures and detect the time signature 
    for each measure. For each chord event (extracted by chordifying each measure), the 
    onset is measured relative to the start of the measure (in quarter-note units).
    
    The ideal positions are defined on a 16th-note grid:
      - For a measure with time signature A/B:
          • Beat duration (in quarter notes) is: beat_duration = 4 / B.
          • The number of beats in the measure is A.
      - Strong beats are at offsets: 0, 1×beat_duration, 2×beat_duration, …, (A-1)×beat_duration.
      - Half-beats are at: (i + 0.5)×beat_duration for i = 0,1,...,A-1.
      - Eighth-note subdivisions are at: (i + 0.25)×beat_duration and (i + 0.75)×beat_duration.
    
    Scoring is defined as follows (using a 4/4 example):
      - Score 0 if the chord is at the beginning of the measure (offset 0),
      - Score 1 if the chord is on any other strong beat (e.g., 1.0, 2.0, or 3.0),
      - Score 2 if the chord is on a half-beat (e.g., 0.5, 1.5, etc.),
      - Score 3 if the chord is on an eighth-note subdivision (e.g., 0.25, 0.75, etc.),
      - Score 4 for all other positions.
    
    The final CBS is the average of these scores over all chord events.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        chord_part_index (int): Index of the part containing chords.
        tolerance (float): Tolerance (in quarter notes) to consider an event "on" an ideal position.
    
    Returns:
        float: The average CBS score.
    """
    # Parse the MIDI file using music21.
    score = converter.parse(midi_file_path)
    
    # Select the chord part.
    try:
        chord_part = score.parts[chord_part_index]
    except IndexError:
        raise ValueError(f"Chord part index {chord_part_index} is out of range.")
    
    # Extract measures from the chord part.
    measures = chord_part.getElementsByClass('Measure')
    if not measures:
        raise ValueError("No measures found in the chord part.")
    
    def ideal_positions_for_measure(ts):
        """
        Given a TimeSignature object, return three lists:
          - strong: positions (in quarter notes) of strong beats,
          - half: positions for half-beats,
          - eighth: positions for eighth-note subdivisions.
        """
        num_beats = ts.numerator
        beat_duration = 4 / ts.denominator  # Duration of one beat in quarter notes.
        measure_duration = num_beats * beat_duration
        
        strong = [i * beat_duration for i in range(num_beats)]
        half = [i * beat_duration + 0.5 * beat_duration for i in range(num_beats)]
        
        eighth = []
        for i in range(num_beats):
            pos1 = i * beat_duration + 0.25 * beat_duration
            pos2 = i * beat_duration + 0.75 * beat_duration
            if pos1 < measure_duration:
                eighth.append(pos1)
            if pos2 < measure_duration:
                eighth.append(pos2)
        return strong, half, eighth
    
    scores = []
    for m in measures:
        # Get the time signature for the measure. Use the first if there are multiple.
        ts_list = m.getTimeSignatures()
        ts = ts_list[0] if ts_list else meter.TimeSignature('4/4')
        
        strong, half, eighth = ideal_positions_for_measure(ts)
        
        # Chordify the measure to collapse simultaneous events into chord events.
        m_chords = m.chordify().recurse().getElementsByClass('Chord')
        for ch in m_chords:
            offset = ch.offset  # offset within the measure (in quarter notes)
            # New scoring: 
            # 0 if exactly at beginning of measure, 1 if on other strong beats,
            # 2 if on half-beat, 3 if on eighth subdivision, 4 otherwise.
            if abs(offset - strong[0]) <= tolerance:
                score_val = 0
            elif any(abs(offset - pos) <= tolerance for pos in strong[1:]):
                score_val = 1
            elif any(abs(offset - pos) <= tolerance for pos in half):
                score_val = 2
            elif any(abs(offset - pos) <= tolerance for pos in eighth):
                score_val = 3
            else:
                score_val = 4
            scores.append(score_val)
    
    return np.mean(scores) if scores else 0.0
    
cbs = compute_CBS(midi_file_path)
print('CBS:', cbs)

####

def extract_chords_from_midi(midi_file_path, track_index=1, grouping_threshold=0.05):
    """
    Extract chords from a MIDI file from the specified track.
    Group together notes that start within 'grouping_threshold' seconds.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        track_index (int): Track index for chords (e.g., 0 = melody, 1 = chords).
        grouping_threshold (float): Time threshold (seconds) to group notes.
    
    Returns:
        List[Tuple[int]]: List of chords (each chord is a tuple of sorted pitch classes).
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    if track_index >= len(pm.instruments):
        raise ValueError(f"Track index {track_index} is out of range for file {midi_file_path}.")
    
    chord_instrument = pm.instruments[track_index]
    sorted_notes = sorted(chord_instrument.notes, key=lambda note: note.start)
    
    chords = []
    current_chord_notes = []
    current_time = None

    for note in sorted_notes:
        if current_time is None:
            current_time = note.start
            current_chord_notes.append(note)
        else:
            if note.start - current_time <= grouping_threshold:
                current_chord_notes.append(note)
            else:
                chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
                chords.append(chord_pitch_classes)
                current_chord_notes = [note]
                current_time = note.start

    if current_chord_notes:
        chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
        chords.append(chord_pitch_classes)
    
    return chords



def levenshtein_distance(seq1, seq2):
    """
    Compute the Levenshtein edit distance between two sequences of chords.
    
    Each chord is assumed to be represented as a tuple of pitch classes,
    e.g., (0, 4, 7) for a C major chord.
    
    Parameters:
        seq1 (List[Tuple[int]]): First chord sequence (e.g., generated chords).
        seq2 (List[Tuple[int]]): Second chord sequence (e.g., ground truth chords).
        
    Returns:
        int: The Levenshtein edit distance.
    """
    n = len(seq1)
    m = len(seq2)
    
    # Initialize a (n+1) x (m+1) matrix.
    D = [[0] * (m + 1) for _ in range(n + 1)]
    
    # Initialize first row and column.
    for i in range(n + 1):
        D[i][0] = i
    for j in range(m + 1):
        D[0][j] = j
    
    # Fill in the rest of the matrix.
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # If the chord tuples are identical, no substitution cost.
            if seq1[i - 1] == seq2[j - 1]:
                cost = 0
            else:
                cost = 1
            D[i][j] = min(
                D[i - 1][j] + 1,      # deletion
                D[i][j - 1] + 1,      # insertion
                D[i - 1][j - 1] + cost  # substitution
            )
    return D[n][m]

def normalized_levenshtein_distance(seq1, seq2):
    """
    Compute a normalized Levenshtein distance between two chord sequences.
    
    The normalization divides the edit distance by the length of the ground truth sequence.
    
    Parameters:
        seq1 (List[Tuple[int]]): First chord sequence.
        seq2 (List[Tuple[int]]): Second chord sequence.
    
    Returns:
        float: Normalized edit distance.
    """
    if len(seq2) == 0:
        return float('inf') if len(seq1) > 0 else 0.0
    return levenshtein_distance(seq1, seq2) / len(seq2)



#extract chords from midi
real_file_path = './data/results/midis/bart_0.8/ChordSymbolTokenizer/mid_0002_real.mid'
gen_file_path = './data/results/midis/bart_0.8/ChordSymbolTokenizer/mid_0002_gen.mid'

real_seq = extract_chords_from_midi(real_file_path)
gen_seq = extract_chords_from_midi(gen_file_path)

LD = normalized_levenshtein_distance(real_seq, gen_seq)
print('LD:', LD)


[2.0, 0.5, 1.5, 2.0, 2.0, 2.0, 0.5, 1.5]
HRHE: 1.0397177708449181
HRC: 3
CBS: 0.2222222222222222
LD: 0.8


In [1]:
import os
import math
import glob
import numpy as np
import pretty_midi
import pandas as pd
from music21 import converter

#####################################
# 1. Chord Extraction Functions
#####################################

def extract_chords_from_midi(midi_file_path, track_index=1, grouping_threshold=0.05):
    """
    Extract chords from a MIDI file from the specified track.
    Group together notes that start within 'grouping_threshold' seconds.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        track_index (int): Track index for chords (e.g., 0 = melody, 1 = chords).
        grouping_threshold (float): Time threshold (seconds) to group notes.
    
    Returns:
        List[Tuple[int]]: List of chords (each chord is a tuple of sorted pitch classes).
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    if track_index >= len(pm.instruments):
        raise ValueError(f"Track index {track_index} is out of range for file {midi_file_path}.")
    
    chord_instrument = pm.instruments[track_index]
    sorted_notes = sorted(chord_instrument.notes, key=lambda note: note.start)
    
    chords = []
    current_chord_notes = []
    current_time = None

    for note in sorted_notes:
        if current_time is None:
            current_time = note.start
            current_chord_notes.append(note)
        else:
            if note.start - current_time <= grouping_threshold:
                current_chord_notes.append(note)
            else:
                chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
                chords.append(chord_pitch_classes)
                current_chord_notes = [note]
                current_time = note.start

    if current_chord_notes:
        chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
        chords.append(chord_pitch_classes)
    
    return chords

def extract_chords_with_times_from_midi(midi_file_path, track_index=1, grouping_threshold=0.05):
    """
    Extract chords from the specified track of a MIDI file along with their onset times.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        track_index (int): Track index for chords.
        grouping_threshold (float): Time threshold to group notes.
    
    Returns:
        List[Tuple[float, Tuple[int]]]: A list of (onset_time, chord) pairs.
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    if track_index >= len(pm.instruments):
        raise ValueError(f"Track index {track_index} is out of range for file {midi_file_path}.")
    
    chord_instrument = pm.instruments[track_index]
    sorted_notes = sorted(chord_instrument.notes, key=lambda note: note.start)
    
    chords_with_times = []
    current_chord_notes = []
    current_time = None

    for note in sorted_notes:
        if current_time is None:
            current_time = note.start
            current_chord_notes.append(note)
        else:
            if note.start - current_time <= grouping_threshold:
                current_chord_notes.append(note)
            else:
                chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
                chords_with_times.append((current_time, chord_pitch_classes))
                current_chord_notes = [note]
                current_time = note.start

    if current_chord_notes:
        chord_pitch_classes = tuple(sorted({n.pitch % 12 for n in current_chord_notes}))
        chords_with_times.append((current_time, chord_pitch_classes))
    
    return chords_with_times

#####################################
# 2. Tonal Centroid via Lookup (Harte et al., 2006)
#####################################

def tonal_centroid(notes):
    """
    Compute the 6-D tonal centroid for a set of pitch classes using lookup tables.
    The centroid is computed by averaging three 2-D vectors:
      - Fifths (e.g. a perfect fifth is assigned [1.0, 0.0])
      - Minor thirds
      - Major thirds
    The three 2-D vectors are concatenated to form a 6-D vector.
    
    Parameters:
        notes (list of int): List of pitch class numbers (0-11).
    
    Returns:
        np.ndarray: A 6-D numpy array representing the tonal centroid.
    """
    # Lookup dictionaries:
    fifths_lookup = {
        9: [1.0, 0.0],
        2: [math.cos(math.pi / 6.0), math.sin(math.pi / 6.0)],
        7: [math.cos(2.0 * math.pi / 6.0), math.sin(2.0 * math.pi / 6.0)],
        0: [0.0, 1.0],
        5: [math.cos(4.0 * math.pi / 6.0), math.sin(4.0 * math.pi / 6.0)],
        10: [math.cos(5.0 * math.pi / 6.0), math.sin(5.0 * math.pi / 6.0)],
        3: [-1.0, 0.0],
        8: [math.cos(7.0 * math.pi / 6.0), math.sin(7.0 * math.pi / 6.0)],
        1: [math.cos(8.0 * math.pi / 6.0), math.sin(8.0 * math.pi / 6.0)],
        6: [0.0, -1.0],
        11: [math.cos(10.0 * math.pi / 6.0), math.sin(10.0 * math.pi / 6.0)],
        4: [math.cos(11.0 * math.pi / 6.0), math.sin(11.0 * math.pi / 6.0)]
    }
    minor_thirds_lookup = {
        3: [1.0, 0.0],
        7: [1.0, 0.0],
        11: [1.0, 0.0],
        0: [0.0, 1.0],
        4: [0.0, 1.0],
        8: [0.0, 1.0],
        1: [-1.0, 0.0],
        5: [-1.0, 0.0],
        9: [-1.0, 0.0],
        2: [0.0, -1.0],
        6: [0.0, -1.0],
        10: [0.0, -1.0]
    }
    major_thirds_lookup = {
        0: [0.0, 1.0],
        3: [0.0, 1.0],
        6: [0.0, 1.0],
        9: [0.0, 1.0],
        2: [math.cos(7.0 * math.pi / 6.0), math.sin(7.0 * math.pi / 6.0)],
        5: [math.cos(7.0 * math.pi / 6.0), math.sin(7.0 * math.pi / 6.0)],
        8: [math.cos(7.0 * math.pi / 6.0), math.sin(7.0 * math.pi / 6.0)],
        11: [math.cos(7.0 * math.pi / 6.0), math.sin(7.0 * math.pi / 6.0)],
        1: [math.cos(11.0 * math.pi / 6.0), math.sin(11.0 * math.pi / 6.0)],
        4: [math.cos(11.0 * math.pi / 6.0), math.sin(11.0 * math.pi / 6.0)],
        7: [math.cos(11.0 * math.pi / 6.0), math.sin(11.0 * math.pi / 6.0)],
        10: [math.cos(11.0 * math.pi / 6.0), math.sin(11.0 * math.pi / 6.0)]
    }
    
    # Weights for each component:
    r1 = 1.0  # for fifths
    r2 = 1.0  # for minor thirds
    r3 = 0.5  # for major thirds
    
    fifths = [0.0, 0.0]
    minor = [0.0, 0.0]
    major = [0.0, 0.0]
    
    if notes:
        for note in notes:
            for i in range(2):
                fifths[i] += r1 * fifths_lookup[note][i]
                minor[i] += r2 * minor_thirds_lookup[note][i]
                major[i] += r3 * major_thirds_lookup[note][i]
        n = len(notes)
        for i in range(2):
            fifths[i] /= n
            minor[i] /= n
            major[i] /= n
    return np.array(fifths + minor + major)

#####################################
# 3. Metrics
#####################################

def compute_chord_histogram_entropy(chords):
    """
    Compute the Chord Histogram Entropy (CHE) of a chord sequence.
    
    Parameters:
        chords (List[Tuple[int]]): List of chords (each chord is a tuple of pitch classes).
    
    Returns:
        float: Entropy in bits.
    """
    if not chords:
        return 0.0
    histogram = {}
    for chord in chords:
        histogram[chord] = histogram.get(chord, 0) + 1
    total = len(chords)
    entropy = 0.0
    for count in histogram.values():
        p = count / total
        entropy -= p * np.log(p + 1e-6)
    return entropy

def compute_chord_coverage(chords):
    """
    Compute the Chord Coverage (CC) of a chord sequence.
    
    Parameters:
        chords (List[Tuple[int]]): List of chords.
    
    Returns:
        int: Number of unique chords.
    """
    return len(set(chords))

def chord_to_tonal_vector(chord):
    """
    Compute the 6-D tonal vector (tonal centroid) for a chord.
    
    Parameters:
        chord (Tuple[int]): A chord represented as a tuple of pitch classes.
    
    Returns:
        np.ndarray: 6-D tonal vector.
    """
    return tonal_centroid(list(chord))

def chord_tonal_distance(chord1, chord2):
    """
    Compute Euclidean distance between the tonal vectors of two chords.
    
    Parameters:
        chord1, chord2 (Tuple[int]): Chords as tuples of pitch classes.
    
    Returns:
        float: Euclidean distance.
    """
    tv1 = chord_to_tonal_vector(chord1)
    tv2 = chord_to_tonal_vector(chord2)
    return np.linalg.norm(tv1 - tv2)

def compute_chord_tonal_distance(chords):
    """
    Compute average Chord Tonal Distance (CTD) over a sequence of chords.
    
    Parameters:
        chords (List[Tuple[int]]): List of chords.
    
    Returns:
        float: Average CTD.
    """
    if len(chords) < 2:
        return 0.0
    distances = []
    for i in range(1, len(chords)):
        distances.append(chord_tonal_distance(chords[i-1], chords[i]))
    return np.mean(distances)

def compute_ctnctr(midi_file_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    """
    Compute the Chord Tone to Non-Chord Tone Ratio (CTnCTR) for a MIDI file.
    
    For each melody note, the active chord is determined by taking the last chord whose onset
    time is less than or equal to the note's start time. The function counts:
      - Chord tones (nc): melody notes whose pitch class is in the active chord.
      - Non-chord tones (nn): melody notes not in the active chord.
      - Proper non-chord tones (np): for a non-chord tone, the first subsequent note that is different
        is checked; if it is a chord tone and the interval difference is ≤ 2 semitones, it is counted.
    
    The CTnCTR ratio is computed as:
        CTnCTR = (nc + np) / (nc + nn)
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        chord_track_index (int): Index of the track containing chords.
        melody_track_index (int): Index of the track containing the melody.
        grouping_threshold (float): Time threshold for grouping chord events.
    
    Returns:
        float: The CTnCTR ratio.
    """
    # Load the MIDI file.
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    
    if melody_track_index >= len(pm.instruments):
        raise ValueError(f"Melody track index {melody_track_index} out of range for file {midi_file_path}.")
    
    # Sort melody notes by start time.
    melody_notes = sorted(pm.instruments[melody_track_index].notes, key=lambda note: note.start)
    
    # Extract chords along with their onset times.
    # Ensure that extract_chords_with_times_from_midi returns a list of tuples (onset_time, chord)
    # where chord is represented as a list or set of pitch classes.
    chords_with_times = extract_chords_with_times_from_midi(midi_file_path, 
                                                           track_index=chord_track_index, 
                                                           grouping_threshold=grouping_threshold)
    if not chords_with_times:
        return 0.0

    nc = 0  # chord tone count
    nn = 0  # non-chord tone count
    np_ = 0 # proper non-chord tone count

    # Iterate over each melody note.
    for idx, note in enumerate(melody_notes):
        note_time = note.start
        note_pc = note.pitch % 12
        active_chord = None

        # Determine the active chord: the last chord with onset time <= note's start time.
        for chord_time, chord in chords_with_times:
            if chord_time <= note_time:
                active_chord = chord
            else:
                break
        if active_chord is None:
            continue

        # Check if the note is a chord tone.
        if note_pc in active_chord:
            nc += 1
        else:
            nn += 1
            # Loop over subsequent notes until we find the first note that is different.
            for next_note in melody_notes[idx+1:]:
                if next_note.pitch != note.pitch:
                    # Check if this next note is a chord tone and if the interval difference is <= 2 semitones.
                    if (next_note.pitch % 12) in active_chord and abs(next_note.pitch - note.pitch) <= 2:
                        np_ += 1
                    break  # Only consider the first different subsequent note.

    denominator = nc + nn
    if denominator == 0:
        return 0.0
    return (nc + np_) / denominator

def compute_pcs(midi_file_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    """
    Compute the Pitch Consonance Score (PCS) for a MIDI file.
    
    For each melody note, determine the active chord and compute a consonance score based on the interval(s)
    between the melody note and each chord tone (after “raising” the chord tone to the octave immediately below or equal
    to the melody note). Intervals of 0, 3, 4, 7, 8, 9 semitones score 1; a perfect fourth (5 semitones) scores 0;
    all other intervals score -1. Melody notes are grouped into 16th-note windows and the scores are averaged.
    
    Parameters:
        midi_file_path (str): Path to MIDI file.
        chord_track_index (int): Chord track index.
        melody_track_index (int): Melody track index.
        grouping_threshold (float): Time threshold for chord grouping.
    
    Returns:
        float: PCS.
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    chords_with_times = extract_chords_with_times_from_midi(midi_file_path, track_index=chord_track_index, grouping_threshold=grouping_threshold)
    if not chords_with_times:
        return 0.0
    if melody_track_index >= len(pm.instruments):
        raise ValueError(f"Melody track index {melody_track_index} out of range for file {midi_file_path}.")
    melody_notes = sorted(pm.instruments[melody_track_index].notes, key=lambda note: note.start)
    
    # Create 16th-note windows based on beat times (or use a default grid if no beats).
    beats = pm.get_beats()
    windows = []
    if len(beats) < 2:
        t = 0.0
        end_time = pm.get_end_time()
        grid_dur = 0.25  # default duration (adjust as needed)
        while t < end_time:
            windows.append((t, t + grid_dur))
            t += grid_dur
    else:
        for i in range(len(beats) - 1):
            beat_start = beats[i]
            beat_end = beats[i+1]
            window_duration = (beat_end - beat_start) / 4.0
            for j in range(4):
                windows.append((beat_start + j * window_duration, beat_start + (j+1) * window_duration))
    
    def consonance_score(interval):
        if interval in {0, 3, 4, 7, 8, 9}:
            return 1
        elif interval == 5:
            return 0
        else:
            return -1

    window_scores = []
    for (w_start, w_end) in windows:
        notes_in_window = [n for n in melody_notes if w_start <= n.start < w_end]
        if not notes_in_window:
            continue
        note_scores = []
        for note in notes_in_window:
            active_chord = None
            for (chord_time, chord) in chords_with_times:
                if chord_time <= note.start:
                    active_chord = chord
                else:
                    break
            if active_chord is None:
                continue
            chord_tone_scores = []
            for chord_pc in active_chord:
                k = (note.pitch - chord_pc) // 12
                chord_pitch = chord_pc + 12 * k
                if chord_pitch > note.pitch:
                    chord_pitch -= 12
                interval = note.pitch - chord_pitch
                chord_tone_scores.append(consonance_score(interval))
            if chord_tone_scores:
                note_scores.append(np.mean(chord_tone_scores))
        if note_scores:
            window_scores.append(np.mean(note_scores))
    
    return np.mean(window_scores) if window_scores else 0.0

def compute_mctd(midi_file_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    """
    Compute the Melody-chord Tonal Distance (MCTD) for a MIDI file.
    
    For each melody note, determine the active chord, compute the tonal centroids (via lookup)
    for the melody note (singleton) and the chord, compute their Euclidean distance, weight by note duration,
    and average over all melody notes.
    
    Parameters:
        midi_file_path (str): Path to MIDI file.
        chord_track_index (int): Chord track index.
        melody_track_index (int): Melody track index.
        grouping_threshold (float): Time threshold for chord grouping.
    
    Returns:
        float: MCTD.
    """
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    if melody_track_index >= len(pm.instruments):
        raise ValueError(f"Melody track index {melody_track_index} out of range for file {midi_file_path}.")
    melody_notes = sorted(pm.instruments[melody_track_index].notes, key=lambda note: note.start)
    chords_with_times = extract_chords_with_times_from_midi(midi_file_path, track_index=chord_track_index, grouping_threshold=grouping_threshold)
    if not chords_with_times:
        return 0.0
    total_duration = 0.0
    weighted_distance_sum = 0.0
    for note in melody_notes:
        note_duration = note.end - note.start
        active_chord = None
        for (chord_time, chord) in chords_with_times:
            if chord_time <= note.start:
                active_chord = chord
            else:
                break
        if active_chord is None:
            continue
        melody_tc = tonal_centroid([note.pitch % 12])
        chord_tc = tonal_centroid(list(active_chord))
        distance = np.linalg.norm(melody_tc - chord_tc)
        weighted_distance_sum += note_duration * distance
        total_duration += note_duration
    if total_duration == 0:
        return 0.0
    return weighted_distance_sum / total_duration

def compute_HRHE_and_HRC(chords_with_times, quantization=0.05):
    """
    Compute the Harmonic Rhythm Histogram Entropy (HRHE) and Harmonic Rhythm Coverage (HRC)
    from a list of chord events with onset times.
    
    Parameters:
        chords_with_times (List[Tuple[float, chord]]): List of tuples where each tuple contains
            the onset time (in seconds) and a chord (represented as pitch classes).
        quantization (float): Quantization step (in seconds) used to discretize the inter-chord intervals.
    
    Returns:
        tuple: (HRHE, HRC)
            HRHE (float): The entropy (in nats) of the histogram of quantized inter-chord intervals.
            HRC (int): The number of unique quantized interval types.
    """
    if len(chords_with_times) < 2:
        return 0.0, 0

    # Compute inter-onset intervals between successive chord events.
    intervals = []
    for i in range(1, len(chords_with_times)):
        diff = chords_with_times[i][0] - chords_with_times[i-1][0]
        # Quantize the interval (e.g., rounding to the nearest multiple of 'quantization')
        quantized_diff = round(diff / quantization) * quantization
        intervals.append(quantized_diff)
    # print(intervals)
    
    # Build a histogram (frequency count) of the quantized intervals.
    hist = {}
    for diff in intervals:
        hist[diff] = hist.get(diff, 0) + 1

    total = sum(hist.values())
    HRHE = 0.0
    for count in hist.values():
        p = count / total
        HRHE -= p * np.log(p + 1e-6)  # using a small epsilon to avoid log(0)
    
    # HRC: number of unique quantized interval types.
    HRC = len(hist)
    return HRHE, HRC

def compute_CBS(midi_file_path, chord_part_index=1, tolerance=0.05):
    """
    Compute the Chord Beat Strength (CBS) for a MIDI file.
    
    This implementation uses music21 to extract measures and detect the time signature 
    for each measure. For each chord event (extracted by chordifying each measure), the 
    onset is measured relative to the start of the measure (in quarter-note units).
    
    The ideal positions are defined on a 16th-note grid:
      - For a measure with time signature A/B:
          • Beat duration (in quarter notes) is: beat_duration = 4 / B.
          • The number of beats in the measure is A.
      - Strong beats are at offsets: 0, 1×beat_duration, 2×beat_duration, …, (A-1)×beat_duration.
      - Half-beats are at: (i + 0.5)×beat_duration for i = 0,1,...,A-1.
      - Eighth-note subdivisions are at: (i + 0.25)×beat_duration and (i + 0.75)×beat_duration.
    
    Scoring is defined as follows (using a 4/4 example):
      - Score 0 if the chord is at the beginning of the measure (offset 0),
      - Score 1 if the chord is on any other strong beat (e.g., 1.0, 2.0, or 3.0),
      - Score 2 if the chord is on a half-beat (e.g., 0.5, 1.5, etc.),
      - Score 3 if the chord is on an eighth-note subdivision (e.g., 0.25, 0.75, etc.),
      - Score 4 for all other positions.
    
    The final CBS is the average of these scores over all chord events.
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        chord_part_index (int): Index of the part containing chords.
        tolerance (float): Tolerance (in quarter notes) to consider an event "on" an ideal position.
    
    Returns:
        float: The average CBS score.
    """
    # Parse the MIDI file using music21.
    score = converter.parse(midi_file_path)
    
    # Select the chord part.
    try:
        chord_part = score.parts[chord_part_index]
    except IndexError:
        raise ValueError(f"Chord part index {chord_part_index} is out of range.")
    
    # Extract measures from the chord part.
    measures = chord_part.getElementsByClass('Measure')
    if not measures:
        raise ValueError("No measures found in the chord part.")
    
    def ideal_positions_for_measure(ts):
        """
        Given a TimeSignature object, return three lists:
          - strong: positions (in quarter notes) of strong beats,
          - half: positions for half-beats,
          - eighth: positions for eighth-note subdivisions.
        """
        num_beats = ts.numerator
        beat_duration = 4 / ts.denominator  # Duration of one beat in quarter notes.
        measure_duration = num_beats * beat_duration
        
        strong = [i * beat_duration for i in range(num_beats)]
        half = [i * beat_duration + 0.5 * beat_duration for i in range(num_beats)]
        
        eighth = []
        for i in range(num_beats):
            pos1 = i * beat_duration + 0.25 * beat_duration
            pos2 = i * beat_duration + 0.75 * beat_duration
            if pos1 < measure_duration:
                eighth.append(pos1)
            if pos2 < measure_duration:
                eighth.append(pos2)
        return strong, half, eighth
    
    scores = []
    for m in measures:
        # Get the time signature for the measure. Use the first if there are multiple.
        ts_list = m.getTimeSignatures()
        ts = ts_list[0] if ts_list else meter.TimeSignature('4/4')
        
        strong, half, eighth = ideal_positions_for_measure(ts)
        
        # Chordify the measure to collapse simultaneous events into chord events.
        m_chords = m.chordify().recurse().getElementsByClass('Chord')
        for ch in m_chords:
            offset = ch.offset  # offset within the measure (in quarter notes)
            # New scoring: 
            # 0 if exactly at beginning of measure, 1 if on other strong beats,
            # 2 if on half-beat, 3 if on eighth subdivision, 4 otherwise.
            if abs(offset - strong[0]) <= tolerance:
                score_val = 0
            elif any(abs(offset - pos) <= tolerance for pos in strong[1:]):
                score_val = 1
            elif any(abs(offset - pos) <= tolerance for pos in half):
                score_val = 2
            elif any(abs(offset - pos) <= tolerance for pos in eighth):
                score_val = 3
            else:
                score_val = 4
            scores.append(score_val)
    
    return np.mean(scores) if scores else 0.0


#####################################
# Pipeline: Compute All Metrics for a Single MIDI File
#####################################

def compute_all_metrics(midi_file_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    """
    Compute all metrics for a given MIDI file.
    
    Metrics:
      - CHE: Chord Histogram Entropy
      - CC: Chord Coverage
      - CTD: Chord Tonal Distance (lookup-based)
      - HRHE: Harmonic Rhythm Histogram Entropy
      - HRC: Harmonic Rhythm Coverage
      - CBS: Chord Beat Strength
      - CTnCTR: Chord Tone to Non-Chord Tone Ratio
      - PCS: Pitch Consonance Score
      - MCTD: Melody-chord Tonal Distance
    
    Parameters:
        midi_file_path (str): Path to the MIDI file.
        chord_track_index (int): Track index for chords.
        melody_track_index (int): Track index for melody.
        grouping_threshold (float): Time threshold for chord grouping.
    
    Returns:
        dict: Dictionary with keys for each metric.
    """
    chords = extract_chords_from_midi(midi_file_path, track_index=chord_track_index, grouping_threshold=grouping_threshold)
    chords_with_times = extract_chords_with_times_from_midi(midi_file_path, track_index=chord_track_index, grouping_threshold=grouping_threshold)


    metrics = {}
    metrics["CHE"] = compute_chord_histogram_entropy(chords)
    metrics["CC"] = compute_chord_coverage(chords)
    metrics["CTD"] = compute_chord_tonal_distance(chords)
    metrics["HRHE"], metrics["HRC"] = compute_HRHE_and_HRC(chords_with_times, grouping_threshold)
    metrics["CBS"] = compute_CBS(midi_file_path, chord_track_index, grouping_threshold)
    metrics["CTnCTR"] = compute_ctnctr(midi_file_path, chord_track_index, melody_track_index, grouping_threshold)
    metrics["PCS"] = compute_pcs(midi_file_path, chord_track_index, melody_track_index, grouping_threshold)
    metrics["MCTD"] = compute_mctd(midi_file_path, chord_track_index, melody_track_index, grouping_threshold)
    
    return metrics

#####################################
# 4. New Helper: Trim Generated MIDI Using music21
#####################################

def trim_generated_midi_with_music21(real_midi_path, gen_midi_path):
    """
    Use music21 to parse both the real and generated MIDI files and compare
    their number of measures (bars) in the first part. If the generated file
    has more measures than the real file, trim it to contain only the first N measures,
    where N is the number of measures in the real file. The trimmed file is saved
    as a new MIDI file with a '_trimmed' suffix.
    """
    # Parse both files with music21.
    score_real = converter.parse(real_midi_path)
    score_gen = converter.parse(gen_midi_path)
    
    # Get measures from the first part.
    measures_real = list(score_real.parts[0].getElementsByClass('Measure'))
    measures_gen  = list(score_gen.parts[0].getElementsByClass('Measure'))
    
    num_measures_real = len(measures_real)
    num_measures_gen = len(measures_gen)
    
    # If the generated file has the same or fewer measures, no trimming is needed.
    if num_measures_gen <= num_measures_real:
        return gen_midi_path
    
    # Otherwise, keep only the first num_measures_real measures.
    trimmed_gen = score_gen.measures(1, num_measures_real)
    
    # Build the new filename by adding a "_trimmed" suffix.
    base, ext = os.path.splitext(gen_midi_path)
    trimmed_file_path = base + "_trimmed" + ext
    trimmed_gen.write('midi', fp=trimmed_file_path)
    print('Trimmed gen file at:', gen_midi_path)
    return trimmed_file_path


#####################################
# Pipeline: Compute Metrics for Pairs of Real and Generated Files
#####################################

def compute_metrics_for_pairs(folder_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    metrics_by_group = {"real": [], "gen": []}
    files = os.listdir(folder_path)
    pairs = {}
    
    # Identify pairs based on matching filename prefixes (e.g., mid_0000_real.mid and mid_0000_gen.mid).
    for file in files:
        if file.lower().endswith("_real.mid"):
            prefix = file[:-9]  # remove '_real.mid'
            real_file = os.path.join(folder_path, file)
            gen_file = os.path.join(folder_path, prefix + "_gen.mid")
            if os.path.exists(gen_file):
                pairs[prefix] = (real_file, gen_file)
            else:
                print(f"No generated file for {file}")
    
    for prefix, (real_file, gen_file) in pairs.items():
        # Trim the generated file if needed.
        trimmed_gen_file = trim_generated_midi_with_music21(real_file, gen_file)
        try:
            m_real = compute_all_metrics(real_file, chord_track_index, melody_track_index, grouping_threshold)
            m_gen  = compute_all_metrics(trimmed_gen_file, chord_track_index, melody_track_index, grouping_threshold)
        except Exception as e:
            print(f"Error processing pair {prefix}: {e}")
            continue
        metrics_by_group["real"].append(m_real)
        metrics_by_group["gen"].append(m_gen)
    
    # Aggregate metrics per group.
    results = {}
    for group in ["gen", "real"]:
        if metrics_by_group[group]:
            metric_names = metrics_by_group[group][0].keys()
            group_results = {}
            for metric in metric_names:
                values = [m[metric] for m in metrics_by_group[group]]
                group_results[metric] = {
                    "mean": np.mean(values),
                    "std": np.std(values),
                    "count": len(values)
                }
            results[group] = group_results
        else:
            results[group] = {}
    
    for group in ["gen", "real"]:
        print(f"{group.upper()} files:")
        if results[group]:
            for metric, stats in results[group].items():
                print(f"  {metric}: Mean = {stats['mean']:.3f}, Std = {stats['std']:.3f} (n = {stats['count']})")
        else:
            print("  No files found.")
    
    return results


def highlight_closest(series):
    """
    For a metric column (e.g. "CHE_mean"), use the first row as the ground truth
    value and mark (with a lightgreen background) the cell(s) in the generated rows
    that are closest (in absolute difference) to that ground truth.
    """
    metric_columns = ["CHE", "CC", "CTD", "HRHE", "HRC", "CBS", "CTnCTR", "PCS", "MCTD"]
    if series.name not in metric_columns:
        return ['' for _ in series]
    ground_truth = series.iloc[0]
    # Compute differences for generated rows only (rows with index >= 1)
    diffs = series.iloc[1:].apply(lambda x: abs(x - ground_truth))
    if diffs.empty:
        return ['' for _ in series]
    min_diff = diffs.min()
    styles = []
    for i, val in enumerate(series):
        if i == 0:
            styles.append('')  # Do not highlight the ground truth row
        else:
            if abs(val - ground_truth) == min_diff:
                styles.append('background-color: lightgreen')
            else:
                styles.append('')
    return styles


def compute_model_results(model_path, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05):
    """
    Given a model folder (each containing one or more representation subfolders),
    iterate over its representations. For each representation (folder), compute the
    aggregated metrics for paired real and generated MIDI files.
    
    Since the real (ground truth) files are identical across representations,
    the ground truth row is taken from the first representation that yields valid results.
    
    Returns a list of dictionaries (rows) for this model:
      - The first row is the ground truth (with "Model": "Ground Truth (Real)" and no Representation).
      - Subsequent rows are for each representation with generated metrics.
    """
    rows = []
    ground_truth_row = None
    # Iterate over representation folders
    for rep_folder in os.listdir(model_path):
        rep_path = os.path.join(model_path, rep_folder)
        if not os.path.isdir(rep_path):
            continue
        print(f"  Processing representation: {rep_folder}")
        try:
            results = compute_metrics_for_pairs(rep_path, chord_track_index, melody_track_index, grouping_threshold)
        except Exception as e:
            print(f"    Error processing {rep_folder}: {e}")
            continue
        
        # Use the 'real' metrics from the first valid representation as ground truth
        if ground_truth_row is None and results.get("real"):
            gt = results["real"]
            gt_row = { "Representation": "Ground Truth"}
            for metric, stats in gt.items():
                gt_row[f"{metric}"]  = stats.get("mean", np.nan)
                # gt_row[f"{metric}_std"]   = stats.get("std", np.nan)
                # gt_row[f"{metric}_count"] = stats.get("count", np.nan)
            ground_truth_row = gt_row
            rows.append(ground_truth_row)
        
        # Add the generated (gen) metrics row for this representation
        if "gen" in results:
            gen_row = { "Representation": rep_folder}
            for metric, stats in results["gen"].items():
                gen_row[f"{metric}"]  = stats.get("mean", np.nan)
                # gen_row[f"{metric}_std"]   = stats.get("std", np.nan)
                # gen_row[f"{metric}_count"] = stats.get("count", np.nan)
            rows.append(gen_row)
    return rows

def compute_and_save_html_results_by_model(root_folder, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05, output_html="results.html"):
    """
    Iterate over each model folder (under the root folder). For each model, iterate over
    its representation folders to compute aggregated generated metrics (and the common ground truth).
    Then create an HTML table for each model (with a caption showing the model name) and use a
    pandas Styler to highlight (in green) the cell(s) in each metric column (for the generated rows)
    that are closest to the ground truth value.
    
    At the beginning of the HTML file, a legend is added with the full names of the metrics.
    
    Finally, combine all tables into one HTML file.
    """
    html_tables = []
    # Iterate over model folders (each model is a subfolder of root_folder)
    for model_folder in sorted(os.listdir(root_folder)):
        model_path = os.path.join(root_folder, model_folder)
        if not os.path.isdir(model_path):
            continue
        
        print(f"Processing model: {model_folder}")
        # Get all rows for this model
        rows = compute_model_results(model_path, chord_track_index, melody_track_index, grouping_threshold)
        if not rows:
            print(f"  No valid data for model {model_folder}")
            continue
        
        df = pd.DataFrame(rows)
        
        # Define the metric columns we want to highlight (only the _mean columns)
        metric_cols = ["CHE", "CC", "CTD", "HRHE", "HRC", "CBS", "CTnCTR", "PCS", "MCTD"]
        
        # Apply styling: for each metric column, highlight the cell(s) (except the first row) that
        # are closest to the ground truth (first row value)
        styled_df = df.style.apply(highlight_closest, subset=metric_cols)
        # Set a caption with the model name
        styled_df = styled_df.set_caption(f"<h2>Model: {model_folder}</h2>")
        # Render the styled table as HTML and add to our list
        html_tables.append(styled_df.to_html())
    
    # Create a legend block for the metrics
    legend_html = """
    <div style="margin-bottom: 30px;">
      <h2>Metrics Legend</h2>
      <ul>
         <li><strong>CHE</strong>: Chord Histogram Entropy</li>
         <li><strong>CC</strong>: Chord Coverage</li>
         <li><strong>CTD</strong>: Chord Tonal Distance</li>
         <li><strong>HRHE</strong>: Harmonic Rhythm Histogram Entropy</li>
         <li><strong>HRC</strong>: Harmonic Rhythm Coverage</li>
         <li><strong>CBS</strong>: Chord Beat Strength</li>
         <li><strong>CTnCTR</strong>: Chord Tone to Non-Chord Tone Ratio</li>
         <li><strong>PCS</strong>: Pitch Consonance Score</li>
         <li><strong>MCTD</strong>: Melody-chord Tonal Distance</li>
      </ul>
    </div>
    """
    
    # Combine all tables into one HTML string with a header including the legend.
    full_html = f"""
    <html>
      <head>
        <meta charset="utf-8">
        <title>Metrics Results by Model</title>
        <style>
          table {{ border-collapse: collapse; margin-bottom: 30px; }}
          th, td {{ border: 1px solid black; padding: 5px; text-align: center; }}
          caption {{ caption-side: top; font-weight: bold; font-size: 1.2em; margin-bottom: 10px; }}
        </style>
      </head>
      <body>
        {legend_html}
        {"".join(html_tables)}
      </body>
    </html>
    """
    with open(output_html, "w", encoding="utf-8") as f:
        f.write(full_html)
    print(f"HTML results saved to {output_html}")


#####################################
# Main
#####################################

if __name__ == "__main__":
    # Set your folder path here:
    #print(results)
    root_folder = "./results/MIDIs"
    compute_and_save_html_results_by_model(root_folder, chord_track_index=1, melody_track_index=0, grouping_threshold=0.05, output_html="metric_results.html")


Processing model: bart_0.8
  Processing representation: ChordSymbolTokenizer


C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0005_gen.mid


C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
In C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\matplotlib\mpl-data\stylelib\bmh.mplstyle: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
In C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\matplotlib\mpl-data\stylelib\fivethirtyeight.mplstyle: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found 

Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0026_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0107_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0112_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0115_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0296_gen.mid
Error processing pair mid_0327: Track index 1 is out of range for file ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0327_real.mid.
Error processing pair mid_0332: Track index 1 is out of range for file ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0332_real.mid.
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0348_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0395_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbolTokenizer\mid_0463_gen.mid
Trimmed gen file at: ./results/MIDIs\bart_0.8\ChordSymbo

...

Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0025_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0107_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0112_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0115_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0296_gen.mid
Error processing pair mid_0327: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0327_real.mid.
Error processing pair mid_0332: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0332_real.mid.
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0348_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0395_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_0.8\ChordSymbolTokenizer\mid_0463_gen.mid
Trimmed gen file at: ./res

C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0107_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0112_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0115_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0296_gen.mid
Error processing pair mid_0327: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0327_real.mid.
Error processing pair mid_0332: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0332_real.mid.
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0348_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0402_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0463_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.0\ChordSymbolTokenizer\mid_0467_gen.mid
Trimmed gen file at: ./res

C:\Users\dimak\AppData\Roaming\Python\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0025_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0087_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0112_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0115_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0296_gen.mid
Error processing pair mid_0327: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0327_real.mid.
Error processing pair mid_0332: Track index 1 is out of range for file ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0332_real.mid.
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0348_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0463_gen.mid
Trimmed gen file at: ./results/MIDIs\gpt_reg_1.2\ChordSymbolTokenizer\mid_0467_gen.mid
Trimmed gen file at: ./res